# Benchmark gradient

Test the adjoint of `numpy.gradient`

In [1]:
import numpy as np

def gradient_ad(df, *varargs, **kwargs):

    df = np.asanyarray(df)
    N = df.ndim - 1  # number of dimensions

    axes = kwargs.pop('axis', None)
    if axes is None:
        axes = tuple(list(range(N)))
    else:
        axes = _nx.normalize_axis_tuple(axes, N)

    len_axes = len(axes)
    n = len(varargs)
    if n == 0:
        dx = [1.0] * len_axes
    elif n == len_axes or (n == 1 and np.isscalar(varargs[0])):
        dx = list(varargs)
        for i, distances in enumerate(dx):
            if np.isscalar(distances):
                continue
            if len(distances) != f.shape[axes[i]]:
                raise ValueError("distances must be either scalars or match "
                                 "the length of the corresponding dimension")
            diffx = np.diff(dx[i])
            # if distances are constant reduce to the scalar case
            # since it brings a consistent speedup
            if (diffx == diffx[0]).all():
                diffx = diffx[0]
            dx[i] = diffx
        if len(dx) == 1:
            dx *= len_axes
    else:
        raise TypeError("invalid number of arguments")


    edge_order = kwargs.pop('edge_order', 1)
    if kwargs:
        raise TypeError('"{}" are not valid keyword arguments.'.format(
                                                  '", "'.join(kwargs.keys())))
    if edge_order > 2:
        raise ValueError("'edge_order' greater than 2 not supported")


    # create slice objects --- initially all are [:, :, ..., :]
    slice1 = [slice(None)]*N
    slice2 = [slice(None)]*N
    slice3 = [slice(None)]*N
    slice4 = [slice(None)]*N

    otype = df.dtype.char
    if otype not in ['f', 'd', 'F', 'D', 'm', 'M']:
        otype = 'd'
        
    df_ad = np.zeros_like(df[0], dtype=otype)
        
    for i, axis in enumerate(axes):
        uniform_spacing = np.isscalar(dx[i])

        # Numerical differentiation: 2nd order interior
        slice1[axis] = slice(1, -1)
        slice2[axis] = slice(None, -2)
        slice3[axis] = slice(1, -1)
        slice4[axis] = slice(2, None)
        # out[slice1] = a * f[slice2] + b * f[slice3] + c * f[slice4]
        # out[slice1] = (f[slice4] - f[slice2]) / (2. * dx[i])

        dgdf4 = 1./(2*dx[i])
        dgdf2 = -1./(2*dx[i])

        df_ad[slice4] += dgdf4*df[i][slice1]
        df_ad[slice2] += dgdf2*df[i][slice1]
        
#         print i, df_ad[slice4].shape, df[i][slice1].shape, axis, dx[i]
#         print i, df_ad[slice2].shape, df[i][slice1].shape, axis, dx[i]
#         print df_ad


        # Numerical differentiation: 1st order edges
        if edge_order == 1:
            slice1[axis] = 0
            slice2[axis] = 1
            slice3[axis] = 0
            dx_0 = dx[i] if uniform_spacing else dx[i][0]
            # 1D equivalent -- out[0] = (y[1] - y[0]) / (x[1] - x[0])
#             out[slice1] = (y[slice2] - y[slice3]) / dx_0

            dgdf2 = 1./dx_0
            dgdf3 = -1./dx_0

            df_ad[slice2] += dgdf2*df[i][slice1]
            df_ad[slice3] += dgdf3*df[i][slice1]

            slice1[axis] = -1
            slice2[axis] = -1
            slice3[axis] = -2
            dx_n = dx[i] if uniform_spacing else dx[i][-1]
            # 1D equivalent -- out[-1] = (y[-1] - y[-2]) / (x[-1] - x[-2])
#             out[slice1] = (y[slice2] - y[slice3]) / dx_n


            dgdf2 = 1./dx_n
            dgdf3 = -1./dx_n


            df_ad[slice2] += dgdf2*df[i][slice1]
            df_ad[slice3] += dgdf3*df[i][slice1]
        
        # reset the slice object in this dimension to ":"
        slice1[axis] = slice(None)
        slice2[axis] = slice(None)
        slice3[axis] = slice(None)
        slice4[axis] = slice(None)
        
#     print out
    print "hey"
            
    return df_ad

In [2]:
T = np.linspace(298., 850., 100).reshape(10,10)
dT = T/500

def fm_grad(T):
    gradT = np.gradient(T)
    gradT = np.array(gradT)
    cost  = np.sum((gradT[0] - 5.0)**2)
    return cost

def tl_grad(T, dT):
    gradT = np.gradient(T)
    gradT = np.array(gradT)
    gradT2 = np.gradient(T+dT)
    gradT2 = np.array(gradT2)
    dgradTdT = np.array(gradT2) - np.array(gradT)
    dgradTdT = np.gradient(dT)
#     dgradTdTx, dgradTdTy, dgradTdTz = np.gradient(np.ones_like(T), *self.delta[::-1])
    dgradT = dgradTdT*dT
    dcdgradT = 2.0*(gradT[0] - 5.0)
    dc = dcdgradT * dgradT
    return dc.sum()
    

def ad_grad(T):
    gradT = np.gradient(T)
    gradT = np.array(gradT)
    dcdgradT0 = 2.0*(gradT[0] - 5.0)
    dcdgradT1 = np.zeros_like(T)
    dcdgradT2 = np.zeros_like(T)
#     print dcdgradT0
    dT_ad = gradient_ad([dcdgradT0, dcdgradT1, dcdgradT2])
#     dT_ad = gradient_ad(np.ones_like(gradT)) * dcdgradT0
#     print dT_ad
    return dT_ad

In [3]:
from conduction.inversion.grad_ad import gradient_ad

In [4]:
fm0 = fm_grad(T)
fm1 = fm_grad(T+dT)
tl  = tl_grad(T, dT)
ad  = ad_grad(T)
print "finite difference", (fm1 - fm0)
print "tangent linear", tl
print "adjoint", ad.ravel().dot(dT.ravel())

finite difference 1133.29131313
tangent linear 1429.54989899
adjoint 1132.04775023
